In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Data Reading


In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksstorageadls.dfs.core.windows.net/customers")
df.display()

## Dropping the not needed Column

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("domains", split(col("email"), '@')[1])
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))

df.display()

In [0]:
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksstorageadls.dfs.core.windows.net/customers")

In [0]:
%sql
create schema if not exists databricks_cata.silver

In [0]:
%sql
create table if not exists databricks_cata.silver.customers_silver
using delta
location "abfss://silver@databricksstorageadls.dfs.core.windows.net/customers"


In [0]:
%sql
select * from databricks_cata.silver.customers_silver